# PSDDN on YOLOv8n - Google Colab Training

This notebook trains PSDDN (Point-Supervised Deep Detection Network) on ShanghaiTech dataset using free GPU.

**Runtime**: GPU (Runtime → Change runtime type → GPU)

**Estimated Time**: 2-3 hours

## Step 1: Setup Environment

In [ ]:
# Check GPU availability
!nvidia-smi

In [ ]:
# Install dependencies
!pip install ultralytics scipy matplotlib seaborn -q

print("✅ Dependencies installed!")

## Step 2: Clone PSDDN Repository

In [ ]:
# Clone your PSDDN implementation
!git clone https://github.com/zyx-core/PSDDN_YOLOv8.git
%cd PSDDN_YOLOv8

# Clone Ultralytics YOLOv8
!git clone https://github.com/ultralytics/ultralytics.git ultralytics_repo

print("✅ Repository cloned!")

## Step 3: Download ShanghaiTech Dataset

In [ ]:
# Download ShanghaiTech dataset
!mkdir -p data
%cd data
!git clone https://github.com/desenzhou/ShanghaiTechDataset.git shanghaitech
%cd ..

print("✅ Dataset downloaded!")

## Step 4: Convert Annotations

In [ ]:
# Convert MAT annotations to JSON
!python scripts/convert_shanghaitech.py \
    --mat-dir data/shanghaitech/part_B/train_data/ground-truth \
    --output data/shanghaitech/part_B_train_annotations.json

!python scripts/convert_shanghaitech.py \
    --mat-dir data/shanghaitech/part_B/test_data/ground-truth \
    --output data/shanghaitech/part_B_test_annotations.json

print("✅ Annotations converted!")

## Step 5: Generate Pseudo GT

In [ ]:
!python scripts/pseudo_gt_init.py \
    --annotations data/shanghaitech/part_B_train_annotations.json \
    --images data/shanghaitech/part_B/train_data/images \
    --output data/shanghaitech/labels/train \
    --img-width 1024 \
    --img-height 768 \
    --format json

print("✅ Pseudo GT generated!")

## Step 6: Create Curriculum Folds

In [ ]:
!python scripts/curriculum_sorting.py \
    --labels data/shanghaitech/labels/train \
    --output data/shanghaitech/folds

print("✅ Folds created!")

## Step 7: Create Config

In [ ]:
%%writefile data/shanghaitech/shanghaitech_partB.yaml
path: /content/PSDDN_YOLOv8/data/shanghaitech
train: part_B/train_data/images
val: part_B/test_data/images
names:
  0: head
nc: 1

## Step 8: Train 🚀

In [ ]:
import sys
sys.path.insert(0, '/content/PSDDN_YOLOv8/ultralytics_repo')
sys.path.insert(0, '/content/PSDDN_YOLOv8/scripts')

from psddn_trainer import train_psddn

train_psddn(
    data_yaml='data/shanghaitech/shanghaitech_partB.yaml',
    folds_dir='data/shanghaitech/folds',
    model='yolov8n.pt',
    epochs=100,
    batch=16,
    imgsz=640,
    project='runs/psddn',
    name='colab_exp',
    device=0,
)

print("✅ Training done!")

## Step 9: Inference

In [ ]:
!python scripts/inference.py \
    --model runs/psddn/colab_exp/weights/best.pt \
    --images data/shanghaitech/part_B/test_data/images \
    --output results \
    --gt data/shanghaitech/part_B_test_annotations.json

## Step 10: View Results

In [ ]:
import json

with open('results/metrics.json') as f:
    m = json.load(f)

print(f"MAE: {m['MAE']:.2f}")
print(f"MSE: {m['MSE']:.2f}")
print(f"AP: {m['AP']:.4f}")

## Step 11: Download Results

In [ ]:
!zip -r results.zip runs results

from google.colab import files
files.download('results.zip')